In [1]:
import numpy as np
import cv2
import torch
from functions import show_tensor
from generator import define_G
from discriminator import define_D
import losses
from replay_pool import ReplayPool
from tqdm import tqdm
import os
from moving_average import moving_average

device = torch.device("cuda")

def make_generator():
    """Create a generator with the specified architecture"""
    return define_G(
        input_nc=3,
        output_nc=3,
        ngf=64,
        netG="global",
        norm="instance",
        n_downsample_global=3,
        n_blocks_global=9,
        n_local_enhancers=1,
        n_blocks_local=3
    ).to(device)

def save_checkpoint(model, checkpoint_dir, epoch):
    """Save model checkpoint"""
    import datetime
    out_file = f"epoch_{epoch}_{datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')}.pt"
    out_file = os.path.join(checkpoint_dir, out_file)
    
    try:
        torch.save({
            "G1": model.G1_ema.state_dict(),
            "G2": model.G2_ema.state_dict(),
            "G3": model.G3_ema.state_dict(),
            "D1": model.D1.state_dict(),
            "D2": model.D2.state_dict(),
            "D3": model.D3.state_dict()
        }, out_file)
        print(f"Saved to {out_file}")
    except Exception as e:
        print(f"Error saving the file: {e}")

def process_loss(log, losses):
    """
    Process and aggregate loss values, updating the log dictionary
    
    Args:
        log (dict): Dictionary to store accumulated loss values
        losses (dict): Dictionary of current loss values to process
        
    Returns:
        torch.Tensor: Sum of all loss values
    """
    total_loss = 0
    
    # Add each loss component to the total and update the log
    for name, value in losses.items():
        if isinstance(value, torch.Tensor):
            total_loss += value
            if name not in log:
                log[name] = 0
            log[name] += value.item()
        else:
            total_loss += value
            if name not in log:
                log[name] = 0
            log[name] += value
            
    return total_loss



class CascadeGAN:
    def __init__(self, device, generators, discriminators, optimizers, replay_pools, losses):
        self.device = device
        
        # Generators
        self.G1 = generators['G1']
        self.G2 = generators['G2']
        self.G3 = generators['G3']
        self.G1_ema = generators['G1_ema']
        self.G2_ema = generators['G2_ema']
        self.G3_ema = generators['G3_ema']
        
        # Discriminators
        self.D1 = discriminators['D1']
        self.D2 = discriminators['D2']
        self.D3 = discriminators['D3']
        
        # Optimizers
        self.G1_optim = optimizers['G1']
        self.G2_optim = optimizers['G2']
        self.G3_optim = optimizers['G3']
        self.D1_optim = optimizers['D1']
        self.D2_optim = optimizers['D2']
        self.D3_optim = optimizers['D3']
        
        # Replay pools
        self.replay_pool1 = replay_pools['pool1']
        self.replay_pool2 = replay_pools['pool2']
        self.replay_pool3 = replay_pools['pool3']
        
        # Loss functions
        self.criterionGAN = losses['GAN']
        self.criterionFeat = losses['Feat']
        self.criterionVGG = losses['VGG']

    def calc_G1_losses(self, img_A, img_A_mask):
        """Calculate losses for first generator (A → A_mask)"""
        fake_A_mask = self.G1(img_A)
        loss_vgg = criterionVGG(fake_A_mask, img_A_mask)
        
        pred_fake = self.D1(torch.cat([img_A, fake_A_mask], dim=1))
        loss_adv = criterionGAN(pred_fake, 1)
        
        with torch.no_grad():
            pred_true = self.D1(torch.cat([img_A, img_A_mask], dim=1))
            
        loss_adv_feat = self.calc_feature_matching_loss(pred_fake, pred_true)
        
        return {
            "G1_vgg": loss_vgg,
            "G1_adv": loss_adv,
            "G1_adv_feat": 10 * loss_adv_feat
        }, fake_A_mask

    def calc_G2_losses(self, fake_A_mask, img_B_mask):
        """Calculate losses for second generator (A_mask → B_mask)"""
        fake_B_mask = self.G2(fake_A_mask.detach())
        loss_vgg = criterionVGG(fake_B_mask, img_B_mask)
        
        pred_fake = self.D2(torch.cat([fake_A_mask.detach(), fake_B_mask], dim=1))
        loss_adv = criterionGAN(pred_fake, 1)
        
        with torch.no_grad():
            pred_true = self.D2(torch.cat([fake_A_mask.detach(), img_B_mask], dim=1))
            
        loss_adv_feat = self.calc_feature_matching_loss(pred_fake, pred_true)
        
        return {
            "G2_vgg": loss_vgg,
            "G2_adv": loss_adv,
            "G2_adv_feat": 10 * loss_adv_feat
        }, fake_B_mask

    def calc_G3_losses(self, fake_B_mask, img_B):
        """Calculate losses for third generator (B_mask → B)"""
        fake_B = self.G3(fake_B_mask.detach())
        loss_vgg = criterionVGG(fake_B, img_B)
        
        pred_fake = self.D3(torch.cat([fake_B_mask.detach(), fake_B], dim=1))
        loss_adv = criterionGAN(pred_fake, 1)
        
        with torch.no_grad():
            pred_true = self.D3(torch.cat([fake_B_mask.detach(), img_B], dim=1))
            
        loss_adv_feat = self.calc_feature_matching_loss(pred_fake, pred_true)
        
        return {
            "G3_vgg": loss_vgg,
            "G3_adv": loss_adv,
            "G3_adv_feat": 10 * loss_adv_feat
        }, fake_B

    def calc_feature_matching_loss(self, pred_fake, pred_true):
        loss_adv_feat = 0
        adv_feats_count = 0
        for d_fake_out, d_true_out in zip(pred_fake, pred_true):
            for l_fake, l_true in zip(d_fake_out[:-1], d_true_out[:-1]):
                loss_adv_feat += criterionFeat(l_fake, l_true)
                adv_feats_count += 1
        return (4/adv_feats_count) * loss_adv_feat if adv_feats_count > 0 else 0

    def calc_D_losses(self, D, real_input, real_target, fake_input, fake_output, name):
        """Calculate discriminator losses"""
        pred_true = D(torch.cat([real_input, real_target], dim=1))
        loss_true = criterionGAN(pred_true, 1)
        
        pred_fake = D(torch.cat([fake_input, fake_output], dim=1))
        loss_false = criterionGAN(pred_fake, 0)
        
        return {
            f"D{name}_true": loss_true,
            f"D{name}_false": loss_false
        }


    def train_step(self, batch):
        img_A = batch['A'].to(self.device)
        img_A_mask = batch['A_mask'].to(self.device)
        img_B_mask = batch['B_mask'].to(self.device)
        img_B = batch['B'].to(self.device)
        
        log = {}
        
        # Stage 1: A → A_mask
        self.G1_optim.zero_grad()
        G1_losses, fake_A_mask = self.calc_G1_losses(img_A, img_A_mask)
        G1_loss = process_loss(log, G1_losses)
        G1_loss.backward()
        self.G1_optim.step()
        moving_average(self.G1, self.G1_ema)
        
        # Detach fake_A_mask to break the graph from stage 1
        fake_A_mask = fake_A_mask.detach()
        
        self.D1_optim.zero_grad()
        fake_data1 = self.replay_pool1.query({
            "input": img_A.detach(),
            "output": fake_A_mask
        })
        D1_losses = self.calc_D_losses(
            self.D1, img_A, img_A_mask,
            fake_data1["input"], fake_data1["output"], "1"
        )
        D1_loss = process_loss(log, D1_losses)
        D1_loss.backward()
        self.D1_optim.step()
        
        # Stage 2: A_mask → B_mask
        self.G2_optim.zero_grad()
        G2_losses, fake_B_mask = self.calc_G2_losses(fake_A_mask, img_B_mask)
        G2_loss = process_loss(log, G2_losses)
        G2_loss.backward()
        self.G2_optim.step()
        moving_average(self.G2, self.G2_ema)
        
        # Detach fake_B_mask to break the graph from stage 2
        fake_B_mask = fake_B_mask.detach()
        
        self.D2_optim.zero_grad()
        fake_data2 = self.replay_pool2.query({
            "input": fake_A_mask,
            "output": fake_B_mask
        })
        D2_losses = self.calc_D_losses(
            self.D2, fake_A_mask, img_B_mask,
            fake_data2["input"], fake_data2["output"], "2"
        )
        D2_loss = process_loss(log, D2_losses)
        D2_loss.backward()
        self.D2_optim.step()
        
        # Stage 3: B_mask → B
        self.G3_optim.zero_grad()
        G3_losses, fake_B = self.calc_G3_losses(fake_B_mask, img_B)
        G3_loss = process_loss(log, G3_losses)
        G3_loss.backward()
        self.G3_optim.step()
        moving_average(self.G3, self.G3_ema)
        
        self.D3_optim.zero_grad()
        fake_data3 = self.replay_pool3.query({
            "input": fake_B_mask,
            "output": fake_B.detach()
        })
        D3_losses = self.calc_D_losses(
            self.D3, fake_B_mask, img_B,
            fake_data3["input"], fake_data3["output"], "3"
        )
        D3_loss = process_loss(log, D3_losses)
        D3_loss.backward()
        self.D3_optim.step()
        
        return log, (fake_A_mask, fake_B_mask, fake_B)


    def test(self, test_loader, epoch, iteration, output_dir):
        """Generate test samples"""
        with torch.no_grad():
            batch = next(iter(test_loader))
            img_A = batch['A'].to(self.device)
            
            self.G1_ema.eval()
            fake_A_mask = self.G1_ema(img_A)
            
            self.G2_ema.eval()
            fake_B_mask = self.G2_ema(fake_A_mask)
            
            self.G3_ema.eval()
            fake_B = self.G3_ema(fake_B_mask)
            
            # Create visualization
            pairs = torch.cat([
                img_A,
                fake_A_mask,
                fake_B_mask,
                fake_B,
                batch['B'].to(self.device)
            ], -1)
            
            matrix = []
            for idx in range(img_A.shape[0]):
                img = 255 * (pairs[idx] + 1) / 2
                img = img.cpu().permute(1, 2, 0).clip(0, 255).numpy().astype(np.uint8)
                matrix.append(img)
            
            matrix = np.vstack(matrix)
            matrix = cv2.cvtColor(matrix, cv2.COLOR_RGB2BGR)
            out_file = os.path.join(output_dir, f"{epoch}_{iteration}.jpg")
            cv2.imwrite(out_file, matrix)
            
            self.G1_ema.train()
            self.G2_ema.train()
            self.G3_ema.train()

def train(model, train_loader, test_loader, num_epochs, checkpoint_dir):
    """Training loop"""
    for epoch in range(num_epochs):
        print(f"Training epoch {epoch}...")
        N = 0
        log = {}
        
        pbar = tqdm(train_loader)
        for batch in pbar:
            batch_log, _ = model.train_step(batch)
            
            for k, v in batch_log.items():
                if k not in log:
                    log[k] = 0
                log[k] += v
            
            N += 1
            if (N % 100 == 0) or (N + 1 >= len(train_loader)):
                for i in range(3):
                    model.test(test_loader, epoch, N + i, os.path.join(checkpoint_dir, "images"))
            
            # Update progress bar
            txt = " ".join([f"{k}: {v/N:.3e}" for k, v in log.items()])
            pbar.set_description(txt)
        
        # Save checkpoint
        if (epoch + 1) % 20 == 0:
            save_checkpoint(model, checkpoint_dir, epoch)


# Initialize all three generators and their EMA counterparts
generator1 = make_generator()  # A → A_mask
generator2 = make_generator()  # A_mask → B_mask
generator3 = make_generator()  # B_mask → B

generator1_ema = make_generator()
generator2_ema = make_generator()
generator3_ema = make_generator()

# Initialize EMA weights
with torch.no_grad():
    for g, g_ema in [
        (generator1, generator1_ema),
        (generator2, generator2_ema),
        (generator3, generator3_ema)
    ]:
        for gp, ep in zip(g.parameters(), g_ema.parameters()):
            ep.data = gp.data.detach()

# Initialize discriminators
discriminator1 = define_D(
    input_nc=3 + 3,  # A + A_mask
    ndf=64,
    n_layers_D=3,
    num_D=3,
    norm="instance",
    getIntermFeat=True
).to(device)

discriminator2 = define_D(
    input_nc=3 + 3,  # A_mask + B_mask
    ndf=64,
    n_layers_D=3,
    num_D=3,
    norm="instance",
    getIntermFeat=True
).to(device)

discriminator3 = define_D(
    input_nc=3 + 3,  # B_mask + B
    ndf=64,
    n_layers_D=3,
    num_D=3,
    norm="instance",
    getIntermFeat=True
).to(device)

# Initialize loss functions
criterionGAN = losses.GANLoss(use_lsgan=True).to(device)
criterionFeat = torch.nn.L1Loss().to(device)
criterionVGG = losses.VGGLoss().to(device)

# Initialize replay pools for each stage
replay_pool1 = ReplayPool(10)
replay_pool2 = ReplayPool(10)
replay_pool3 = ReplayPool(10)

# Initialize optimizers
G1_optim = torch.optim.AdamW(generator1.parameters(), lr=1e-4)
G2_optim = torch.optim.AdamW(generator2.parameters(), lr=1e-4)
G3_optim = torch.optim.AdamW(generator3.parameters(), lr=1e-4)

D1_optim = torch.optim.AdamW(discriminator1.parameters(), lr=1e-4)
D2_optim = torch.optim.AdamW(discriminator2.parameters(), lr=1e-4)
D3_optim = torch.optim.AdamW(discriminator3.parameters(), lr=1e-4)


from dataloader import Dataset, print_dataset, plot_batch_samples
# DataLoader setup
batch_size = 8
num_workers = 4

train_dataset = Dataset(images_dir="/media/irfan/New Volume/Dentalverse/code/pix2pix/simplified_pix2pixHD/data/ortho/train", mode="train")
test_dataset = Dataset(images_dir="/media/irfan/New Volume/Dentalverse/code/pix2pix/simplified_pix2pixHD/data/ortho/test", mode="test")

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)
# Print information for both datasets
print_dataset(train_loader, mode="train")
print_dataset(test_loader, mode="test")

# Get and display a batch
batch = next(iter(train_loader))
plot_batch_samples(batch)

# Usage
checkpoint_dir = "/media/irfan/New Volume/Dentalverse/code/pix2pix/simplified_pix2pixHD/checkpoints/ortho"
images_output_dir = os.path.join(checkpoint_dir, "images")
os.makedirs(images_output_dir, exist_ok=True)
device = torch.device("cuda")

# Create the CascadeGAN model
model = CascadeGAN(
    device=device,
    generators={
        'G1': generator1,
        'G2': generator2,
        'G3': generator3,
        'G1_ema': generator1_ema,
        'G2_ema': generator2_ema,
        'G3_ema': generator3_ema
    },
    discriminators={
        'D1': discriminator1,
        'D2': discriminator2,
        'D3': discriminator3
    },
    optimizers={
        'G1': G1_optim,
        'G2': G2_optim,
        'G3': G3_optim,
        'D1': D1_optim,
        'D2': D2_optim,
        'D3': D3_optim
    },
    replay_pools={
        'pool1': replay_pool1,
        'pool2': replay_pool2,
        'pool3': replay_pool3
    },
    losses={
        'GAN': criterionGAN,
        'Feat': criterionFeat,
        'VGG': criterionVGG
    }
)

train(model, train_loader, test_loader, num_epochs=200, checkpoint_dir=checkpoint_dir)


GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace=True)
    (13): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(512, 512, kernel_size

/home/irfan/anaconda3/envs/DiT/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/irfan/anaconda3/envs/DiT/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training epoch 0...


G1_vgg: 7.452e-01 G1_adv: 1.071e+00 G1_adv_feat: 4.430e+00 D1_true: 9.385e-01 D1_false: 9.075e-01 G2_vgg: 9.728e-01 G2_adv: 1.259e+00 G2_adv_feat: 7.831e+00 D2_true: 7.732e-01 D2_false: 6.996e-01 G3_vgg: 8.126e-01 G3_adv: 1.249e+00 G3_adv_feat: 7.644e+00 D3_true: 7.218e-01 D3_false: 6.696e-01: 100%|██████████| 533/533 [01:19<00:00,  6.68it/s]


Training epoch 1...


G1_vgg: 6.063e-01 G1_adv: 8.386e-01 G1_adv_feat: 2.676e+00 D1_true: 7.147e-01 D1_false: 7.267e-01 G2_vgg: 9.948e-01 G2_adv: 1.286e+00 G2_adv_feat: 7.671e+00 D2_true: 5.328e-01 D2_false: 4.914e-01 G3_vgg: 8.183e-01 G3_adv: 1.146e+00 G3_adv_feat: 8.110e+00 D3_true: 5.521e-01 D3_false: 5.188e-01: 100%|██████████| 533/533 [01:19<00:00,  6.67it/s]


Training epoch 2...


...

Training epoch 29...


G1_vgg: 2.186e-01 G1_adv: 7.831e-01 G1_adv_feat: 1.474e+00 D1_true: 7.480e-01 D1_false: 7.482e-01 G2_vgg: 8.649e-01 G2_adv: 1.311e+00 G2_adv_feat: 9.317e+00 D2_true: 5.392e-01 D2_false: 4.886e-01 G3_vgg: 7.413e-01 G3_adv: 1.246e+00 G3_adv_feat: 1.154e+01 D3_true: 5.630e-01 D3_false: 3.888e-01: 100%|██████████| 533/533 [01:18<00:00,  6.76it/s]


Training epoch 30...


G1_vgg: 2.147e-01 G1_adv: 7.867e-01 G1_adv_feat: 1.457e+00 D1_true: 7.480e-01 D1_false: 7.450e-01 G2_vgg: 8.990e-01 G2_adv: 1.348e+00 G2_adv_feat: 9.479e+00 D2_true: 5.364e-01 D2_false: 4.807e-01 G3_vgg: 7.398e-01 G3_adv: 1.223e+00 G3_adv_feat: 1.120e+01 D3_true: 5.361e-01 D3_false: 4.141e-01:  21%|██        | 112/533 [00:16<01:00,  6.92it/s]


KeyboardInterrupt: 